# Logistic Regression Consulting Project

## Binary Customer Churn

A marketing agency has many customers that use their service to produce ads for the client/customer websites. They've noticed that they have quite a bit of churn in clients. They basically randomly assign account managers right now, but want you to create a machine learning model that will help predict which customers will churn (stop buying their service) so that they can correctly assign the customers most at risk to churn an account manager. Luckily they have some historical data, can you help them out? Create a classification algorithm that will help classify whether or not a customer churned. Then the company can test this against incoming data for future customers to predict which customers will churn and assign them an account manager.

The data is saved as customer_churn.csv. Here are the fields and their definitions:

    Name : Name of the latest contact at Company
    Age: Customer Age
    Total_Purchase: Total Ads Purchased
    Account_Manager: Binary 0=No manager, 1= Account manager assigned
    Years: Totaly Years as a customer
    Num_sites: Number of websites that use the service.
    Onboard_date: Date that the name of the latest contact was onboarded
    Location: Client HQ Address
    Company: Name of Client Company
    
Once you've created the model and evaluated it, test out the model on some new data (you can think of this almost like a hold-out set) that your client has provided, saved under new_customers.csv. The client wants to know which customers are most likely to churn given this data (they don't have the label yet).

# EDA

In [1]:
import pyspark
spark = pyspark.sql.SparkSession.builder.appName("foo").getOrCreate()

In [2]:
data = spark.read.csv(
    "/home/jovyan/udemy_pyspark/resources/Spark_for_Machine_Learning/Logistic_Regression/customer_churn.csv",
    inferSchema=True,
    header=True,
)
data.printSchema()
data.describe().show()
data.head()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: string (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- Churn: integer (nullable = true)

+-------+-------------+-----------------+-----------------+------------------+-----------------+------------------+-------------------+--------------------+--------------------+-------------------+
|summary|        Names|              Age|   Total_Purchase|   Account_Manager|            Years|         Num_Sites|       Onboard_date|            Location|             Company|              Churn|
+-------+-------------+-----------------+-----------------+------------------+-----------------+------------------+-------------------+--------------------+--------------------+----------------

Row(Names='Cameron Williams', Age=42.0, Total_Purchase=11066.8, Account_Manager=0, Years=7.22, Num_Sites=8.0, Onboard_date='2013-08-30 07:00:40', Location='10265 Elizabeth Mission Barkerburgh, AK 89518', Company='Harvey LLC', Churn=1)

In [3]:
# new data without label
test_data = spark.read.csv(
    "/home/jovyan/udemy_pyspark/resources/Spark_for_Machine_Learning/Logistic_Regression/new_customers.csv",
    inferSchema=True,
    header=True,
)
test_data.printSchema()
test_data.describe().show()
test_data.head()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: string (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)

+-------+-------------+------------------+-----------------+------------------+-----------------+------------------+-------------------+--------------------+----------------+
|summary|        Names|               Age|   Total_Purchase|   Account_Manager|            Years|         Num_Sites|       Onboard_date|            Location|         Company|
+-------+-------------+------------------+-----------------+------------------+-----------------+------------------+-------------------+--------------------+----------------+
|  count|            6|                 6|                6|                 6|                6|     

Row(Names='Andrew Mccall', Age=37.0, Total_Purchase=9935.53, Account_Manager=1, Years=7.71, Num_Sites=8.0, Onboard_date='2011-08-29 18:37:54', Location='38612 Johnny Stravenue Nataliebury, WI 15717-8316', Company='King Ltd')

# EDA

In [4]:
data.columns

['Names',
 'Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites',
 'Onboard_date',
 'Location',
 'Company',
 'Churn']

In [5]:
# There are 873 different companies among 900 -> rather useless feature 
data.select("Company").distinct().count()

873

# Feature extraction

## Extract a relevant feature from onboard_date

In [6]:
data.head()

Row(Names='Cameron Williams', Age=42.0, Total_Purchase=11066.8, Account_Manager=0, Years=7.22, Num_Sites=8.0, Onboard_date='2013-08-30 07:00:40', Location='10265 Elizabeth Mission Barkerburgh, AK 89518', Company='Harvey LLC', Churn=1)

In [7]:
from pyspark.sql.functions import to_date, current_date, months_between
data = data.withColumn(
    "months_since_last_contact",
    months_between(current_date(), to_date(data.Onboard_date)),
)
data.head()

Row(Names='Cameron Williams', Age=42.0, Total_Purchase=11066.8, Account_Manager=0, Years=7.22, Num_Sites=8.0, Onboard_date='2013-08-30 07:00:40', Location='10265 Elizabeth Mission Barkerburgh, AK 89518', Company='Harvey LLC', Churn=1, months_since_last_contact=100.93548387)

In [8]:
from pyspark.sql.functions import to_timestamp, to_date, current_date
data.na.drop().count()

900

## Train

In [9]:
data.columns

['Names',
 'Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites',
 'Onboard_date',
 'Location',
 'Company',
 'Churn',
 'months_since_last_contact']

In [10]:
selected_columns = [
# 'Names',
 'Age',
 'Total_Purchase',
 # 'Account_Manager',
 'Account_Manager_Cat', # one hot encoded
 'Years',
 'Num_Sites',
 # 'Onboard_date',
 # 'Location',
 # 'Company',
 # 'Churn', # this is the 
 'months_since_last_contact'
]

In [11]:
from pyspark.ml.feature import (
    VectorAssembler,
    VectorIndexer,
    OneHotEncoder,
    StringIndexer,
)
from pyspark.ml.classification import LogisticRegression

In [12]:
pipeline = pyspark.ml.Pipeline(stages=[
        OneHotEncoder(inputCol="Account_Manager", outputCol="Account_Manager_Cat"),
        VectorAssembler(inputCols=selected_columns, outputCol="features"),
        LogisticRegression(labelCol="Churn"),
])      

In [13]:
train_data, val_data = data.randomSplit([0.7,0.3])

In [14]:
model = pipeline.fit(train_data)

# Evaluate

In [15]:
results = model.transform(val_data)
results.head()

Row(Names='Aaron Meyer', Age=45.0, Total_Purchase=9598.03, Account_Manager=0, Years=5.0, Num_Sites=7.0, Onboard_date='2010-07-17 03:30:38', Location='35821 Bailey Skyway Alexisstad, NH 81472', Company='Steele, Bates and Lane', Churn=0, months_since_last_contact=138.35483871, Account_Manager_Cat=SparseVector(1, {0: 1.0}), features=DenseVector([45.0, 9598.03, 1.0, 5.0, 7.0, 138.3548]), rawPrediction=DenseVector([4.8744, -4.8744]), probability=DenseVector([0.9924, 0.0076]), prediction=0.0)

In [16]:
from pyspark.ml.evaluation import (
    BinaryClassificationEvaluator, MulticlassClassificationEvaluator
)
BinaryClassificationEvaluator(
    # rawPredictionCol="rawPrediction", # returns 0.9411  - this is the default!
    # rawPredictionCol="prediction", # returns 0.78
    labelCol="Churn",
    # metricName="areaUnderROC",
).evaluate(results)

0.9167542016806723

In [17]:
MulticlassClassificationEvaluator(
    predictionCol="prediction",
    labelCol="Churn",
    # metricName="f1", # default
).evaluate(results)

0.8708992416034669

# Predict on test set

In [18]:
test_data = spark.read.csv(
    "/home/jovyan/udemy_pyspark/resources/Spark_for_Machine_Learning/Logistic_Regression/new_customers.csv",
    inferSchema=True,
    header=True,
)
test_data.printSchema()
test_data.head()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: string (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)



Row(Names='Andrew Mccall', Age=37.0, Total_Purchase=9935.53, Account_Manager=1, Years=7.71, Num_Sites=8.0, Onboard_date='2011-08-29 18:37:54', Location='38612 Johnny Stravenue Nataliebury, WI 15717-8316', Company='King Ltd')

In [19]:
test_data = test_data.withColumn(
    "months_since_last_contact",
    months_between(current_date(), to_date(test_data.Onboard_date)),
)
test_data.head()

Row(Names='Andrew Mccall', Age=37.0, Total_Purchase=9935.53, Account_Manager=1, Years=7.71, Num_Sites=8.0, Onboard_date='2011-08-29 18:37:54', Location='38612 Johnny Stravenue Nataliebury, WI 15717-8316', Company='King Ltd', months_since_last_contact=124.96774194)

In [20]:
test_results = model.transform(test_data)
test_results.select("Company", "prediction").show()

+----------------+----------+
|         Company|prediction|
+----------------+----------+
|        King Ltd|       0.0|
|   Cannon-Benson|       1.0|
|Barron-Robertson|       1.0|
|   Sexton-Golden|       1.0|
|        Wood LLC|       0.0|
|   Parks-Robbins|       1.0|
+----------------+----------+



Now you could assign Account managers to the following companies:
- Cannon-Brenson
- Barron-Robertson
- Sexton-Golden
- Parks-Robbins